# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Notebook part 1: Scraping website data and formatting it into a pandas dataframe


In this notbook I will show how to scrape table data from the following website:  
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
    

### First we have to import the following Libraries 

In [2]:
import pandas as pd # the pandas package
import numpy as np # the numpy package
from bs4 import BeautifulSoup # for the webscraping we use the package beautifulsoup.
import requests # for retreiving the web data

### Downloading the website data

Now the libraries are loaded we use 'requests' to download the raw data from the webpage.  
After retreiving the raw data, we have to parse it and search for the table we need.  
Finaly we will select the table containing the postal codes we need and convert it to a pandas dataframe


In [3]:
# defining the URL
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# now using requests lets retrieve the xml raw data
response = requests.get(wiki_url,'lxml')

In [4]:
# read the data and find the correct table using the class label in the HTML code, and read the html code into a list of pandas data frames called 'tables'
soup = BeautifulSoup(response.text,'html.parser')
postal_table =soup.find('table', {'class': "wikitable sortable"})
tabels = pd.read_html(str(postal_table))

In [5]:
# read the correct table out of the list of dataframes 
df= tabels[0]
# correct the column name Neighbourhood to Neighborhood
df.rename(columns={"Neighbourhood": "Neighborhood"},inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Cleaning up the table

### The above table needs to be cleaned in order to:

- Ignore cells with a borough haveing a value 'Not assigned'. In order to do so we remove them from the table end therefore exclude them for future processing
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- If more than one neighborhood exist in one postal code area and there are multiple entries in the table, rows have to be combined

### Ignore cells with a borough has a value 'Not assigned'

In [6]:
# the below code drops any row containing the value "Not assigned" in the Borough column and reset the indexing of the table
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Update the name of the Neighborhood to name of borough if having the value "Not assigned"

In [7]:
# in this loop all the neighborhoods containing value 'Not assigned' will be updated by the value of the Borough
i=[]
for i in df.loc[df['Neighborhood']=='Not assigned'].index:
    df.iloc[i]['Neighborhood'] = df.iloc[i]['Borough']

### Combining rows if multiple postal code entries exist in the table

First we perform a simple test to see if there are any rows we have to combine.

In [8]:
# test if there are postal codes listed multiple times and have to be to combined into one row
tel = df['Postal Code'].value_counts(sort=True)
tel.head()

M3K    1
M5V    1
M8Z    1
M4G    1
M3L    1
Name: Postal Code, dtype: int64

Conclusion: There are no duplicate postal code entries in the table to combine
In case (for future use) we need a way to combine them, the below code can be used.

In [9]:
# This code creates a new dataframe containing all the postal codes and groups multiple entries of the same postal codes 
# with the neighborhoods separated by a ','

df_combined = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df_combined.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Saving the data

In [10]:
# safe the data frame to an Excel file for later use.
df.to_excel("./Data/dataframe.xlsx",index=False)  

### Checking for the format of the table using the .shape function and print the first twelve rows of the dataframe

In [11]:
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
# display the shape
df.shape

(103, 3)